This notebook is to try to implement the Quadratic Energy algorithm that is mentioned in this paper: https://www.ifaamas.org/Proceedings/aamas2019/pdfs/p1722.pdf. For the other solving algorithms of the acceptibility semantics, since they follows a similar pipeline, it's easy to development them by starting from what we will have below. 

To validate the algorithm correctness, we use the stock example mentioned in this notebook: https://github.com/nicopotyka/Uncertainpy/blob/master/examples/gradual/gradual_argumentation_examples.ipynb.

In [1]:
import numpy as np

# A simple example: BAG containing three arguments {a, b, c} 
# links: attack(a, b), attack(c, b), support(c, a)
vs = np.array([[0, 0, 1], [-1, 0, -1], [0, 0, 0]])
weights = np.array([0.5, 0.8, 0.2])
step_size = 10e-2
epsilon = 10e-4
max_iter = 1000000

In [2]:
# implementation of the aggretation and influence algorithms
# as well as the corresponding update process of single step

def aggreg_sum(v_parent: np.ndarray, v_strength: np.ndarray):
    if v_parent.size != v_strength.size:
        raise ValueError(f"Size of input vectors should be the same, but now {v_parent.size} and {v_strength.size}")
    return v_parent @ v_strength

def infl_pmax(aggreg_strength: float, weight: float, p: int=2, k: float=1):
    h = lambda x: max(0, x) ** p / (1 + max(0, x) ** p)
    return weight * (1 - h(-aggreg_strength / k) + h(aggreg_strength / k))
    
def compute_delta(vs_parent: np.ndarray, v_strength: np.ndarray, v_weight: np.ndarray):
    result = []
    for i in range(vs_parent.shape[0]):
        aggreg_s = aggreg_sum(v_parent=vs_parent[i], v_strength=v_strength) 
        updated_strength = infl_pmax(aggreg_strength=aggreg_s, weight=v_weight[i]) 
        result.append(updated_strength)
    return np.array(result) - v_strength

In [4]:
# RK4 approximator implementation
def rk4(v_strength: np.ndarray, weights: np.ndarray, step_size: float, max_iter: int):
    for step in range(max_iter):
        k1 = compute_delta(vs_parent=vs, v_strength=v_strength, v_weight=weights)
        k2 = compute_delta(vs_parent=vs, v_strength=v_strength + 0.5 * step_size * k1, v_weight=weights)
        k3 = compute_delta(vs_parent=vs, v_strength=v_strength + 0.5 * step_size * k2, v_weight=weights)
        k4 = compute_delta(vs_parent=vs, v_strength=v_strength + step_size * k3, v_weight=weights)
        delta = step_size * (k1 + 2 * k2 + 2 * k3 + k4) / 6 
        v_strength += delta
        step += 1
        if abs(delta).max() < epsilon:
            break 
    return step, v_strength

In [5]:
# Result is as expected,
# s1 increase slightly, as the weight of argument 3 is relatively small, 
# s2 decreased significantly, as both 1 and 3 attack it.
# s3 doesn't change, as no support or attack is targeting on 3.

import copy

v_strength = copy.deepcopy(weights) # weight as initial strength
print(f"initial stength: {v_strength}")

step, v_strength = rk4(v_strength=v_strength, weights=weights, step_size=step_size, max_iter=max_iter)

print(f"Convergence reached at step {step}, final strenght: {v_strength}")

initial stength: [0.5 0.8 0.2]
Convergence reached at step 35, final strenght: [0.51865005 0.53650836 0.2       ]
